# 메모리

- 챗봇이 이전사람과 대화한 내용을 기억하고 있는 

- 모델자체에는 메모리가 없다.

## ConversationBufferMemory

- 단점은 대화 내용이 길어질수록 사이즈가 커짐. (모든 대화내용을 저장)

In [10]:
from langchain.memory import ConversationBufferMemory

# return_messages True 이면 AIMessage 도 응답해준다
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input" : "Hi!"}, {"output" : "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

## ConversationBufferWindowMemory
-  대화의 특정부분만 저장. 최근 5개의 대화내용 저장 하는 그런식


In [11]:
from langchain.memory import ConversationBufferWindowMemory
# 대화의 특정부분만 저장. 최근 5개의 대화내용 저장 하는 그런식

memory = ConversationBufferWindowMemory(return_messages=True, k=4)

def add_message(input, output):
    memory.save_context({"input": input}, {"ouput" : output})


In [12]:

for i in range(1, 10):
    add_message(i,f"{i}입니다")


In [14]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='6'),
  AIMessage(content='6입니다'),
  HumanMessage(content='7'),
  AIMessage(content='7입니다'),
  HumanMessage(content='8'),
  AIMessage(content='8입니다'),
  HumanMessage(content='9'),
  AIMessage(content='9입니다')]}

## ConversationSummaryMemory
- 메모리 실행하는데 비용이 든다
- 처음 실행에는 보다 많은 토큰이 들지만, 대화내용이 점점 많아지면 많아질수록 효과가 좋아진다.

In [15]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)



def add_message(input, output):
    memory.save_context({"input": input}, {"ouput" : output})

def get_history():
    return memory.load_memory_variables({})

In [16]:
add_message("Hi I'm Nicolas, I live in South Korea", "Wow taht is so cool!")

In [18]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [19]:
get_history()

{'history': 'The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool. The human mentions that South Korea is pretty, to which the AI responds by expressing a desire to go there.'}

## ConversationSummaryBufferMemory
- limit 에 도달할때까지 buffer에 넣고 그리고 summary 를 하는 메모리

In [22]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=150, return_messages=True)


In [23]:
add_message("Hi I'm Nicolas, I live in South Korea", "Wow taht is so cool!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow taht is so cool!')]}

In [24]:
add_message("South Korea is so pretty", "I wish I could go!!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow taht is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [28]:
add_message("South Korea is so pretty", "I wish I could go!!")
get_history()

{'history': [SystemMessage(content="The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds with enthusiasm, saying that it thinks it's cool and expresses a desire to visit because South Korea is so pretty. The AI adds that it wishes it could go too and emphasizes its longing to visit South Korea. The human agrees, saying that South Korea is indeed beautiful."),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessa

## ConversationKGMemory
- 지식그래프 를 만들어낼 수 있음

In [44]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationKGMemory(llm=llm, return_messages=True)




In [45]:
add_message("Hi I'm Nicolas, I live in South Korea", "Wow taht is so cool!")


In [46]:
add_message("Nicoals likes kimchi", "Wow taht is so cool!")


In [37]:
memory.load_memory_variables({"input": "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea.')]}

In [48]:
memory.load_memory_variables({"input": "what does Nicoals like?"})

{'history': [SystemMessage(content='On Nicoals: Nicoals likes kimchi.')]}

## Memory On LLMChain

In [55]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You:

"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)


In [56]:

chain.predict(question="My name is Nico")

chain.predict(question="I live in Seoul")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human: My name is Nico
    You:



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human: I live in Seoul
    You:



> Finished chain.


"That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?"

In [54]:
memory.load_memory_variables({})

{'history': 'System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them. Nico mentions that they live in Seoul. The AI responds by providing information about Seoul, including its vibrant culture, modern architecture, delicious food, and popular attractions. The AI also mentions that Seoul is a hub for technology and entertainment, with numerous K-pop concerts and events taking place in the city.'}

In [57]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?
    Human: What is my name?
    You:



> Finished chain.


'Your name is Nico.'

## Chat Based Momory





In [58]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt= prompt,
    verbose=True
)


In [59]:

chain.predict(question="My name is Nico")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Nico

> Finished chain.


'Hello Nico! How can I assist you today?'

In [60]:

chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul

> Finished chain.


"That's great, Nico! Seoul is a vibrant and bustling city with a rich cultural heritage. How can I help you with anything related to Seoul?"

In [61]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great, Nico! Seoul is a vibrant and bustling city with a rich cultural heritage. How can I help you with anything related to Seoul?
Human: What is my name?

> Finished chain.


'Your name is Nico.'

## LCEL Based Memory


In [64]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough 
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_hsitory=load_memory) | prompt | llm

chain.invoke({
    # "chat_history" : load_memory,
    "question" : "My name is Nico"
})






{'question': 'My name is Nico'}


KeyError: 'chat_history'